# Aligning Data to Darwin Core - Sampling Event with Measurement or Fact using Python
Matt Biddle

November 9, 2020

# General information about this notebook
This notebook was created for the IOOS DMAC Code Sprint Biological Data Session
The data in this notebook were created specifically as an example and meant solely to be
illustrative of the process for aligning data to the biological data standard - Darwin Core.
These data should not be considered actually occurrences of species and any measurements
are also contrived. This notebook is meant to provide a step by step process for taking
original data and aligning it to Darwin Core

This notebook is a python implementation of the R notebook [IOOS_DMAC_DataToDWC_Notebook_event.R](https://github.com/ioos/bio_data_guide/blob/master/Standardizing%20Marine%20Biological%20Data/datasets/example_script_with_fake_data/IOOS_DMAC_DataToDwC_Notebook_event.R)

In [1]:
import pandas as pd
import pyworms # pip install git+git://github.com/iobis/pyworms.git
import numpy as np
import uuid
import csv

Read in the raw data file.

In [2]:
file = 'data/MadeUpDataForBiologicalDataTraining.csv'
df = pd.read_csv(file, header=[0])

First we need to to decide if we will provide an occurrence only version of the data or
a sampling event with measurement or facts version of the data. Occurrence only is easier
to create. It's only one file to produce. However, several pieces of information will be
left out if we choose that option. If we choose to do sampling event with measurement or
fact we'll be able to capture all of the data in the file creating a lossless version.
Here we decide to use the sampling event option to include as much information as we can.

First let's create the eventID and occurrenceID in the original file so that information
can be reused for all necessary files down the line.

In [3]:
df['eventID'] = df[['region', 'station', 'transect']].apply(lambda x: '_'.join(x.astype(str)), axis=1)
df['occurrenceID'] = uuid.uuid4()

# uuid for each row
#for index, row in df.iterrows():
#    df.loc[index, 'occurrenceID'] = uuid.uuid4()

# Event file

We will need to create three separate files to comply with the sampling event format.
We'll start with the event file but we only need to include the columns that are relevant
to the event file.

In [4]:
event = df[['date', 'lat', 'lon', 'region', 'station', 'transect', 'depth', 'bottom type', 'eventID']].copy()

Next we need to rename any columns of data that match directly to Darwin Core. We know
this based on our crosswalk spreadsheet CrosswalkToDarwinCore.csv

In [5]:
event['decimalLatitude'] = event['lat']
event['decimalLongitude'] = event['lon']
event['minimumDepthInMeters'] = event['depth']
event['maximumDepthInMeters'] = event['depth']
event['habitat'] = event['bottom type']
event['island'] = event['region']

Let's see how it looks:

In [6]:
event.head()

,date,lat,lon,region,station,transect,depth,bottom type,eventID,decimalLatitude,decimalLongitude,minimumDepthInMeters,maximumDepthInMeters,habitat,island
0,7/16/2004,18.29788,-64.79451,St. John,250,1,25,shallow reef flat,St. John_250_1,18.29788,-64.79451,25,25,shallow reef flat,St. John
1,7/16/2004,18.29788,-64.79451,St. John,250,1,25,shallow reef flat,St. John_250_1,18.29788,-64.79451,25,25,shallow reef flat,St. John
2,7/16/2004,18.29788,-64.79451,St. John,250,1,25,shallow reef flat,St. John_250_1,18.29788,-64.79451,25,25,shallow reef flat,St. John
3,7/16/2004,18.29788,-64.79451,St. John,250,1,25,shallow reef flat,St. John_250_1,18.29788,-64.79451,25,25,shallow reef flat,St. John
4,7/16/2004,18.29788,-64.79451,St. John,250,2,35,complex back reef,St. John_250_2,18.29788,-64.79451,35,35,complex back reef,St. John


We need to convert the date to ISO format

In [7]:
event['eventDate'] = pd.to_datetime(
                            event['date'],
                            format='%m/%d/%Y',
                            utc=True)

We will also have to add any missing required fields

In [8]:
event['basisOfRecord'] = 'HumanObservation'
event['geodeticDatum'] = 'EPSG:4326 WGS84'

Then we'll remove any columns that we no longer need to clean things up a bit.

In [9]:
event.drop(
    columns=['date', 'lat', 'lon', 'region', 'station', 'transect', 'depth', 'bottom type'],
    inplace=True)

We have too many repeating rows of information. We can pare this down using eventID which
is a unique identifier for each sampling event in the data- which is six, three transects
per site.

In [10]:
event.drop_duplicates(
    subset='eventID',
    inplace=True)

event.head(6)

,eventID,decimalLatitude,decimalLongitude,minimumDepthInMeters,maximumDepthInMeters,habitat,island,eventDate,basisOfRecord,geodeticDatum
0,St. John_250_1,18.29788,-64.79451,25,25,shallow reef flat,St. John,2004-07-16 00:00:00+00:00,HumanObservation,EPSG:4326 WGS84
4,St. John_250_2,18.29788,-64.79451,35,35,complex back reef,St. John,2004-07-16 00:00:00+00:00,HumanObservation,EPSG:4326 WGS84
8,St. John_250_3,18.29788,-64.79451,85,85,deep reef,St. John,2004-07-16 00:00:00+00:00,HumanObservation,EPSG:4326 WGS84
12,St. John_356_1,18.27609,-64.75740,28,28,complex back reef,St. John,2004-07-17 00:00:00+00:00,HumanObservation,EPSG:4326 WGS84
16,St. John_356_2,18.27609,-64.75740,16,16,shallow reef flat,St. John,2004-07-17 00:00:00+00:00,HumanObservation,EPSG:4326 WGS84
20,St. John_356_3,18.27609,-64.75740,90,90,deep reef,St. John,2004-07-17 00:00:00+00:00,HumanObservation,EPSG:4326 WGS84


Finally we write out the event file

In [11]:
event.to_csv(
    'MadeUpData_event_frompy.csv',
    header=True,
    index=False,
    date_format='%Y-%m-%d')

# Occurrence file
Next we need to create the occurrence file. We start by creating the dataframe.

In [12]:
occurrence = df[['scientific name', 'eventID', 'occurrenceID', 'percent cover']].copy()

Then we'll rename the columns that align directly with Darwin Core.

In [13]:
occurrence['scientificName'] = occurrence['scientific name']

Finally we'll add required information that's missing.

In [14]:
occurrence['occurrenceStatus'] = np.where(occurrence['percent cover'] == 0, 'absent', 'present')

## Taxonomic Name Matching
A requirement for OBIS is that all scientific names match to the World Register of
Marine Species (WoRMS) and a scientificNameID is included. A scientificNameID looks
like this "urn:lsid:marinespecies.org:taxname:275730" with the last digits after
the colon being the WoRMS aphia ID. We'll need to go out to WoRMS to grab this
information.

Create a lookup table of unique scientific names

In [15]:
lut_worms = pd.DataFrame(
    columns=['scientificName'],
    data=occurrence['scientificName'].unique())

Add the columns that we can grab information from WoRMS including the required scientificNameID.

In [16]:
headers = ['acceptedname', 'acceptedID', 'scientificNameID', 'kingdom', 'phylum',
           'class', 'order', 'family', 'genus', 'scientificNameAuthorship', 'taxonRank']

for head in headers:
    lut_worms[head] = ''

Taxonomic lookup using the library [pyworms](https://github.com/iobis/pyworms)

In [17]:
for index, row in lut_worms.iterrows():
    print('Searching for scientific name = %s' % row['scientificName'])
    resp = pyworms.aphiaRecordsByMatchNames(row['scientificName'])[0][0]
    lut_worms.loc[index, 'acceptedname'] = resp['valid_name']
    lut_worms.loc[index, 'acceptedID'] = resp['valid_AphiaID']
    lut_worms.loc[index, 'scientificNameID'] = resp['lsid']
    lut_worms.loc[index, 'kingdom'] = resp['kingdom']
    lut_worms.loc[index, 'phylum'] = resp['phylum']
    lut_worms.loc[index, 'class'] = resp['class']
    lut_worms.loc[index, 'order'] = resp['order']
    lut_worms.loc[index, 'family'] = resp['family']
    lut_worms.loc[index, 'genus'] = resp['genus']
    lut_worms.loc[index, 'scientificNameAuthorship'] = resp['authority']
    lut_worms.loc[index, 'taxonRank'] = resp['rank']

Searching for scientific name = Acropora cervicornis
Searching for scientific name = Madracis auretenra
Searching for scientific name = Mussa angulosa
Searching for scientific name = Siderastrea radians


Merge the lookup table of unique scientific names back with the occurrence data.

In [18]:
occurrence = pd.merge(occurrence, lut_worms, how='left', on='scientificName')

We're going to remove any unnecessary columns to clean up the file

In [19]:
occurrence.drop(
    columns=['scientific name', 'percent cover'],
    inplace=True)

Quick look at what we have before we write out the file

In [20]:
occurrence.head()

,eventID,occurrenceID,scientificName,occurrenceStatus,acceptedname,acceptedID,scientificNameID,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,taxonRank
0,St. John_250_1,d8705645-c74a-4f05-9f1c-2e97cbde2fb6,Acropora cervicornis,absent,Acropora cervicornis,206989,urn:lsid:marinespecies.org:taxname:206989,Animalia,Cnidaria,Anthozoa,Scleractinia,Acroporidae,Acropora,"(Lamarck, 1816)",Species
1,St. John_250_1,d8705645-c74a-4f05-9f1c-2e97cbde2fb6,Madracis auretenra,present,Madracis auretenra,430664,urn:lsid:marinespecies.org:taxname:430664,Animalia,Cnidaria,Anthozoa,Scleractinia,Pocilloporidae,Madracis,"Locke, Weil & Coates, 2007",Species
2,St. John_250_1,d8705645-c74a-4f05-9f1c-2e97cbde2fb6,Mussa angulosa,present,Mussa angulosa,216135,urn:lsid:marinespecies.org:taxname:216135,Animalia,Cnidaria,Anthozoa,Scleractinia,Faviidae,Mussa,"(Pallas, 1766)",Species
3,St. John_250_1,d8705645-c74a-4f05-9f1c-2e97cbde2fb6,Siderastrea radians,absent,Siderastrea radians,207517,urn:lsid:marinespecies.org:taxname:207517,Animalia,Cnidaria,Anthozoa,Scleractinia,Siderastreidae,Siderastrea,"(Pallas, 1766)",Species
4,St. John_250_2,d8705645-c74a-4f05-9f1c-2e97cbde2fb6,Acropora cervicornis,absent,Acropora cervicornis,206989,urn:lsid:marinespecies.org:taxname:206989,Animalia,Cnidaria,Anthozoa,Scleractinia,Acroporidae,Acropora,"(Lamarck, 1816)",Species


Write out the file.

In [21]:
# sort the columns on scientificName
occurrence.sort_values('scientificName', inplace=True)
# reorganize column order to be consistent with R example:
columns = ["scientificName","eventID","occurrenceID","occurrenceStatus","acceptedname","acceptedID",
           "scientificNameID","kingdom","phylum","class","order","family","genus","scientificNameAuthorship",
           "taxonRank"]

occurrence.to_csv(
    "MadeUpData_Occurrence_frompy.csv",
    header=True,
    index=False,
    quoting=csv.QUOTE_ALL,
    columns=columns)

 All done with occurrence!

# Measurement Or Fact
The last file we need to create is the measurement or fact file. For this we need to
combine all of the measurements or facts that we want to include making sure to include
IDs from the BODC NERC vocabulary where possible.

Temperature

In [22]:
temperature = df[['eventID', 'temperature', 'date']].copy()
temperature['occurrenceID'] = ''
temperature['measurementType'] = 'temperature'
temperature['measurementTypeID'] = 'http://vocab.nerc.ac.uk/collection/P25/current/WTEMP/'
temperature['measurementValue'] = temperature['temperature']
temperature['measurementUnit'] = 'Celsius'
temperature['measurementUnitID'] = 'http://vocab.nerc.ac.uk/collection/P06/current/UPAA/'
temperature['measurementAccuracy'] = 3
temperature['measurementDeterminedDate'] = pd.to_datetime(temperature['date'],
                                                          format='%m/%d/%Y',
                                                          utc=True)
temperature['measurementMethod'] = ''
temperature.drop(
    columns=['temperature', 'date'],
    inplace=True)

Rugosity

In [23]:
rugosity = df[['eventID', 'rugosity', 'date']].copy()
rugosity['occurrenceID'] = ''
rugosity['measurementType'] = 'rugosity'
rugosity['measurementTypeID'] = ''
rugosity['measurementValue'] = rugosity['rugosity'].map('{:,.6f}'.format)
rugosity['measurementUnit'] = ''
rugosity['measurementUnitID'] = ''
rugosity['measurementAccuracy'] = ''
rugosity['measurementDeterminedDate'] = pd.to_datetime(rugosity['date'],
                                                       format='%m/%d/%Y',
                                                       utc=True)
rugosity['measurementMethod'] = ''
rugosity.drop(
    columns=['rugosity', 'date'],
    inplace=True)

Percent cover

In [24]:
percent_cover = df[['eventID', 'occurrenceID', 'percent cover', 'date']].copy()
percent_cover['measurementType'] = 'Percent Cover'
percent_cover['measurementTypeID'] = 'http://vocab.nerc.ac.uk/collection/P01/current/SDBIOL10/'
percent_cover['measurementValue'] = percent_cover['percent cover']
percent_cover['measurementUnit'] = 'Percent/100m^2'
percent_cover['measurementUnitID'] = ''
percent_cover['measurementAccuracy'] = 5
percent_cover['measurementDeterminedDate'] = pd.to_datetime(percent_cover['date'],
                                                          format='%m/%d/%Y',
                                                          utc=True)
percent_cover['measurementMethod'] = ''
percent_cover.drop(
    columns=['percent cover', 'date'],
    inplace=True)

Concatenate all measurements or facts together.

In [25]:
measurementorfact = pd.concat([temperature, rugosity, percent_cover])

Let's check to see what it looks like

In [26]:
measurementorfact.head(50)

,eventID,occurrenceID,measurementType,measurementTypeID,measurementValue,measurementUnit,measurementUnitID,measurementAccuracy,measurementDeterminedDate,measurementMethod
0,St. John_250_1,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,25.2,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
1,St. John_250_1,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,25.2,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
2,St. John_250_1,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,25.2,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
3,St. John_250_1,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,25.2,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
4,St. John_250_2,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,24.8,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
5,St. John_250_2,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,24.8,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
6,St. John_250_2,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,24.8,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
7,St. John_250_2,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,24.8,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
8,St. John_250_3,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,23.1,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,
9,St. John_250_3,,temperature,http://vocab.nerc.ac.uk/collection/P25/current...,23.1,Celsius,http://vocab.nerc.ac.uk/collection/P06/current...,3,2004-07-16 00:00:00+00:00,


Write measurement or fact file

In [27]:
measurementorfact.to_csv('MadeUpDate_mof_frompy.csv',
                         index=False,
                         header=True,
                         date_format='%Y-%m-%d')